# CC3001 2022-2 - Tarea 6: Ordenación usando árboles de búsqueda -- Leonardo Rikhardsson


## Profesores: Nelson Baloian, Jérémy Barbay, Benjamín Bustos, Patricio Poblete

Según las condiciones de su sección, pueden entregar un link a su tarea en `colab` o subir un archivo en el formato Jupyter NoteBook (`.ipynb`). Puede crear todas las funciones auxiliares que requiera para implementar las funciones solicitadas.

# El Problema



Se desea implementar y comparar métodos de ordenación, que reciban un arreglo y ordenen su contenido en orden ascendente.

Estos métodos deben usar el siguiente enfoque:

* Crear un árbol de búsqueda, inicialmente vacío
* Insertar uno por uno todos los elementos del arreglo en al árbol
* Recorrer el árbol en inorden para ir llenando el arreglo con los elementos ordenados

Usted debe implementar dos métodos distintos de ordenación usando este esquema. La diferencia está en el tipo de árbol de búsqueda utilizado. Estos deben ser:

1. Un árbol de búsqueda binaria, sin balance (ABB)
1. Un árbol AVL

A continuación, usted debe probar sus métodos con las datos que se le indicará.

Aparte de los usos de funciones de `numpy` que aparecen en el código provisto en el enunciado, en esta tarea ustedes no tendrían necesidad de utilizar funciones de `numpy` o de otras librerías (pero se espera que utilicen código relevante de los apuntes del curso).


# Su solución

In [ ]:
import numpy as np

class NodoiABB:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der

    def inordenABB(self):
        yield from self.izq.inordenABB()
        yield self
        yield from self.der.inordenABB()

    def insertABB(self,x):
        assert x!=self.info
        if x<self.info:
            return NodoiABB(self.izq.insertABB(x),self.info,self.der)
        else:
            return NodoiABB(self.izq,self.info,self.der.insertABB(x))

class NodoeABB:
    def __init__(self):
        pass

    def inordenABB(self):
        return
        yield None # no se ejecuta, permite que la función sea un generator

    def insertABB(self,x):
        return NodoiABB(NodoeABB(),x,NodoeABB())


class ArbolABB:
    def __init__(self,raiz=NodoeABB()):
        self.raiz=raiz

    def inordenABB(self):
        yield from self.raiz.inordenABB()

    def insertABB(self,x):
        self.raiz=self.raiz.insertABB(x)

def ordena_ABB(a):
    arbol = ArbolABB() # generamos una lista vacia/arbol vacio
    for i in range(len(a)):
      arbol.insertABB(a[i]) # insertamos cada valor en nuestro arbol
      #for j in arbol.inordenABB():
        #a[i] = j.info
    return [j.info for j in arbol.inordenABB()] # nos retorna la nueva lista

In [ ]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
        self.height=1+max(izq.height,der.height)

    def inorden(self):
        yield from self.izq.inorden()
        yield self
        yield from self.der.inorden()

    def right_rotation(self):
        return(Nodoi(self.izq.izq,
                     self.izq.info,
                     Nodoi(self.izq.der,self.info,self.der)))

    def left_rotation(self):
        return(Nodoi(Nodoi(self.izq,self.info,self.der.izq),
                     self.der.info,
                     self.der.der))

    def insert(self,x):
        assert x!=self.info
        if x<self.info:
            p=Nodoi(self.izq.insert(x),self.info,self.der)
            if p.izq.height>p.der.height+1:
                if x<p.izq.info: # inserción exterior
                    p=p.right_rotation()
                else: # inserción interior
                    p=Nodoi(p.izq.left_rotation(),p.info,p.der).right_rotation()
        else: # x>self.info, simétrico del anterior
            p=Nodoi(self.izq,self.info,self.der.insert(x))
            if p.der.height>p.izq.height+1:
                if x>p.der.info: # inserción exterior
                    p=p.left_rotation()
                else: # inserción interior
                    p=Nodoi(p.izq,p.info,p.der.right_rotation()).left_rotation()
        return p

class Nodoe:
    def __init__(self):
        self.height=0

    def inorden(self):
        return
        yield None # no se ejecuta, permite que la función sea un generator

    def insert(self,x):
        return Nodoi(Nodoe(),x,Nodoe())

class ArbolAVL:
    def __init__(self,raiz=Nodoe()):
        self.raiz=raiz

    def inorden(self):
        yield from self.raiz.inorden()

    def insert(self,x):
        self.raiz=self.raiz.insert(x)

def ordena_AVL(a):
    arbol = ArbolAVL() # generamos una lista vacia/arbol vacio
    for i in range(len(a)):
      arbol.insert(a[i]) # insertamos cada valor en nuestro arbol
      #for j in arbol.inorden():
        #a[i] = j.info
    return [j.info for j in arbol.inorden()] # nos retorna la nueva lista

# Pruebas

En primer lugar, hay que verificar que sus métodos ordenen correctamente.

In [ ]:
# Chequear que los métodos ordenan correctamente


def chequea_orden(titulo,a):
    print(titulo+":", "Ordenado" if np.all(a[:-1]<=a[1:]) else "Desordenado")

print("Probando ordena_ABB:")
a = np.random.random(12)
print(a)
chequea_orden("Antes",a)
a = ordena_ABB(a)
print(a)
chequea_orden("Después",a)

print()

print("Probando ordena_AVL:")
a = np.random.random(12)
print(a)
chequea_orden("Antes",a)
a = ordena_AVL(a)
print(a)
chequea_orden("Después",a)

Probando ordena_ABB:
[0.48645347 0.68743612 0.46434489 0.65009412 0.43054375 0.23622561
 0.98205664 0.47794757 0.78179044 0.47766155 0.41654136 0.70134451]
Antes: Desordenado
[0.23622560575840212, 0.4165413570500711, 0.43054374808605356, 0.4643448856107324, 0.4776615532897165, 0.4779475718925662, 0.4864534689361877, 0.6500941221939059, 0.6874361171584193, 0.7013445075763588, 0.7817904438831286, 0.9820566443777312]
Después: Ordenado

Probando ordena_AVL:
[0.66549285 0.26025042 0.67218667 0.92372957 0.98381167 0.17685484
 0.57626477 0.68286896 0.23068269 0.84495265 0.21434496 0.9686487 ]
Antes: Desordenado
[0.17685484458958167, 0.2143449583601157, 0.2306826888353375, 0.2602504202874152, 0.5762647717154081, 0.6654928455950234, 0.6721866701232339, 0.6828689594142173, 0.8449526478706955, 0.9237295656843847, 0.96864869699612, 0.9838116685622521]
Después: Ordenado


A continuación, hay que comparar la eficiencia de los métodos en dos escenarios:

**Escenario 1:**  Ordenar un arreglo que venga inicialmente ordenado. Para esto genere un arreglo conteniendo los números del 1 al $n$, para $n=200$. En cada caso medir e imprimir el tiempo que demora la ordenación.

In [ ]:
import time
n=200

orig=np.array(range(n))

a=np.array(orig)
ti=time.time_ns()
ordena_ABB(a)
tf=time.time_ns()
print("ordena_ABB:",tf-ti)

a=np.array(orig)
ti=time.time_ns()
ordena_AVL(a)
tf=time.time_ns()
print("ordena_AVL:",tf-ti)

ordena_ABB: 25692782
ordena_AVL: 2648191


**Escenario 2:**  Ordenar un arreglo que contenga elementos aleatorios. Para esto genere un arreglo conteniendo $n$ número al azar, para $n=10$ mil. En cada caso medir e imprimir el tiempo que demora la ordenación. Tenemos cuidado de entregarle al método de ordenación una copia del arreglo, para que ambos puedan funcionar sobre exactamente los mismos datos.

In [ ]:
n=10000

orig=np.random.random(n)

a=np.array(orig)
ti=time.time_ns()
ordena_ABB(a)
tf=time.time_ns()
print("ordena_ABB:",tf-ti)

a=np.array(orig)
ti=time.time_ns()
ordena_AVL(a)
tf=time.time_ns()
print("ordena_AVL:",tf-ti)

ordena_ABB: 226834074
ordena_AVL: 516290539


# Conclusiones

*Compare los resultados obtenidos en ambos escenarios y discuta cuáles serían las causas de las diferencias observadas.*

Por lo que se puede ver, el arreglo AVL es mucho mejor que el ABB y por mucho, cuando se quiere comparar arreglos relativamente largos. Pero no es el caso cuando se llega a "Mega arreglos" con ya mas de 10000 datos, siendo que el ABB es mas rapido, ya que para el arreglo AVL, con tantos datos, de igual forma estos deben ser rotados. Podemos decir que el arbol AVL tiene un maximo (Post data, estuve viendo agregando y eliminando ceros, y claro efecivamente hay un momento que AVL se demora mas que el ABB)(Tambien pude notar que para valores de 1000-4000 los valores son muy dispersos, tanto ABB como AVL son mas rapidos)